# Imports

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
from vaastav.getters import *
from datetime import datetime

from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import torchvision

import torch
import torch.nn as nn

import xgboost as xgb

# Constants

In [3]:
NEXT_GW = 25

# fixtures in sample
FIS = 8

# prediction range
FROM_GW = 25
TO_GW = 29

batch_size = 1

# Device

In [4]:
# device = torch.device('cuda')
# device = torch.device('mps')
device = torch.device('cpu')

# Reading data

In [5]:
upcoming_fixtures = pd.read_csv("data/upcoming/upcoming_fixtures.csv")

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (3,4,5,6,9,10,37,40,41,48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
upcoming_fixtures

,Unnamed: 0,Unnamed: 0.1,Date,Day,Comp,Round,Venue,Squad,Opponent,Start,...,CrdRAvgOverall,xGAvgOverall,xAGAvgOverall,CSAvgOverall,Team ScoreAvgOverall,Opp ScoreAvgOverall,FPLAvgOverall,Finished,Squad H,Squad A
0,0,0.0,2022-08-06,Sat,Premier League,Matchweek 1,Home,Leeds United,Wolves,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN
1,1,1.0,2022-08-13,Sat,Premier League,Matchweek 2,Away,Leeds United,Southampton,Y,...,0.0,0.000000,0.000000,0.000000,2.000000,1.00,2.0,True,NaN,NaN
2,2,2.0,2022-08-21,Sun,Premier League,Matchweek 3,Home,Leeds United,Chelsea,Y,...,0.0,0.050000,0.050000,0.000000,2.000000,1.50,2.0,True,NaN,NaN
3,3,3.0,2022-08-27,Sat,Premier League,Matchweek 4,Away,Leeds United,Brighton,Y,...,0.0,0.366667,0.066667,0.333333,2.333333,1.00,4.0,True,NaN,NaN
4,4,4.0,2022-08-30,Tue,Premier League,Matchweek 5,Home,Leeds United,Everton,Y,...,0.0,0.275000,0.050000,0.250000,1.750000,1.00,3.5,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13874,2723,NaN,2023-04-30,NaN,NaN,NaN,NaN,Tottenham,Liverpool,NaN,...,0.0,0.000000,0.025000,0.250000,1.800000,1.55,1.6,False,Liverpool,Tottenham
13875,2726,NaN,2023-05-06,NaN,NaN,NaN,NaN,Tottenham,Crystal Palace,NaN,...,0.0,0.000000,0.025000,0.250000,1.800000,1.55,1.6,False,Tottenham,Crystal Palace
13876,2724,NaN,2023-05-13,NaN,NaN,NaN,NaN,Tottenham,Aston Villa,NaN,...,0.0,0.000000,0.025000,0.250000,1.800000,1.55,1.6,False,Aston Villa,Tottenham
13877,2727,NaN,2023-05-20,NaN,NaN,NaN,NaN,Tottenham,Brentford,NaN,...,0.0,0.000000,0.025000,0.250000,1.800000,1.55,1.6,False,Tottenham,Brentford


In [6]:
# selecting needed rows
upcoming_fixtures = upcoming_fixtures[(upcoming_fixtures["Finished"] == True) | ((upcoming_fixtures["Finished"] == False) & (upcoming_fixtures["GW"].between(FROM_GW, TO_GW)))]

In [7]:
# upcoming_fixtures[upcoming_fixtures["Name"] == "Erling-Haaland"]

In [8]:
# upcoming_fixtures.loc[9315]

# Dataset

In [9]:
features = ["Min", "Gls", "Ast", "Sh", "SoT", "xG", "npxG", "xAG", "SCA", "GCA", "CS", "CrdY", "CrdR", "Opp rating", "Team Score", "Opp Score", "Was Home"]
to_predict = ["FPL"]

In [10]:
def init_index_table(df):
    names = df["Name"].unique()
    result_dict = {}
    
    current_index = 0
    player_index = 0
    
    for name in names:
        sample_index = 0
        samples_for_name = df[(df["Name"] == name) & (df["Finished"] == False)].shape[0]
        for i in range(samples_for_name):
            result_dict[current_index] = [player_index, sample_index]
            current_index += 1
            sample_index += 1
        player_index += 1
    
    return result_dict

In [11]:
class PandasDataset(Dataset):
    def __init__(self, dataframe, starting_gw, ending_gw):
        self.dataframe = dataframe
        
        self.starting_gw = starting_gw
        self.ending_gw = ending_gw
        self.names = self.dataframe["Name"].unique()
        self.length = self.calculate_length()

        self.index_table = init_index_table(self.dataframe)
        self.transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

    def calculate_length(self):
        length = 0
        for name in self.names:
            length += self.dataframe[(self.dataframe["Name"] == name) & self.dataframe["GW"].between(FROM_GW, TO_GW)].shape[0]
        return length
    
    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        # print(idx)
        player_id, sample_id = self.index_table[idx]
        name = self.names[player_id]

        rows_features = self.dataframe[(self.dataframe["Name"] == name) & (self.dataframe["Finished"] == True)]
        features_item = rows_features.tail(FIS)[features + to_predict].values[:, :-1]
        return features_item

In [12]:
df = upcoming_fixtures

df_general = df
df_gk = df[df["FPL position"] == "GK"]
df_def = df[df["FPL position"] == "DEF"]
df_mid = df[df["FPL position"] == "MID"]
df_fwd = df[df["FPL position"] == "FWD"]

In [13]:
dataset_general = PandasDataset(df_general, FROM_GW, TO_GW)

dataset_gk = PandasDataset(df_gk, FROM_GW, TO_GW)
dataset_def = PandasDataset(df_def, FROM_GW, TO_GW)
dataset_mid = PandasDataset(df_mid, FROM_GW, TO_GW)
dataset_fwd = PandasDataset(df_fwd, FROM_GW, TO_GW)

In [14]:
# df[df["Name"] == "Erling-Haaland"].tail(1)

In [15]:
dataset_general.__len__(), dataset_gk.__len__(), dataset_def.__len__(), dataset_mid.__len__(), dataset_fwd.__len__()

(2516, 162, 905, 1140, 281)

In [16]:
dataloader_general_test = DataLoader(dataset_general, batch_size=batch_size, shuffle=False)

dataloader_gk_test = DataLoader(dataset_gk, batch_size=batch_size, shuffle=False)
dataloader_def_test = DataLoader(dataset_def, batch_size=batch_size, shuffle=False)
dataloader_mid_test = DataLoader(dataset_mid, batch_size=batch_size, shuffle=False)
dataloader_fwd_test = DataLoader(dataset_fwd, batch_size=batch_size, shuffle=False)

In [17]:
len(dataloader_general_test), len(dataloader_gk_test), len(dataloader_def_test), len(dataloader_mid_test), len(dataloader_fwd_test)

(2516, 162, 905, 1140, 281)

In [18]:
# next(iter(dataloader_mid_test))

# Models

In [19]:
class LSTMNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        super(LSTMNetwork, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.transpose(x, 0, 1)
        # x = x.view(-1, batch_size)
        # x = torch.transpose(x, 0, 1)
        # print("FOR", x.shape)
        # print(x.shape)
        # print(x.shape)
        out, _ = self.lstm(x)
        # print(out.shape)
        out = self.fc(out[-1])
        return out

In [20]:
net = LSTMNetwork(1, 20, 2, 1, dropout=0.4).to(device)
net.load_state_dict(torch.load('models/best/best_lstm_general.pt'))
net.double()

LSTMNetwork(
  (lstm): LSTM(1, 20, num_layers=2, dropout=0.4)
  (fc): Linear(in_features=20, out_features=1, bias=True)
)

In [21]:
net_gk = LSTMNetwork(1, 20, 2, 1, dropout=0.4).to(device)
net_gk.load_state_dict(torch.load('models/best/best_lstm_gk.pt', map_location=torch.device(device)))
net_gk.double()

LSTMNetwork(
  (lstm): LSTM(1, 20, num_layers=2, dropout=0.4)
  (fc): Linear(in_features=20, out_features=1, bias=True)
)

In [22]:
net_def = LSTMNetwork(1, 8, 2, 1, dropout=0.4).to(device)
net_def.load_state_dict(torch.load('models/best/best_lstm_def.pt', map_location=torch.device(device)))
net_def.double()

LSTMNetwork(
  (lstm): LSTM(1, 8, num_layers=2, dropout=0.4)
  (fc): Linear(in_features=8, out_features=1, bias=True)
)

In [23]:
net_mid = LSTMNetwork(1, 15, 2, 1, dropout=0.4).to(device)
net_mid.load_state_dict(torch.load('models/best/best_lstm_mid.pt', map_location=torch.device(device)))
net_mid.double()

LSTMNetwork(
  (lstm): LSTM(1, 15, num_layers=2, dropout=0.4)
  (fc): Linear(in_features=15, out_features=1, bias=True)
)

In [24]:
net_fwd = LSTMNetwork(1, 8, 2, 1, dropout=0.4).to(device)
net_fwd.load_state_dict(torch.load('models/best/best_lstm_fwd.pt', map_location=torch.device(device)))
net_fwd.double()

LSTMNetwork(
  (lstm): LSTM(1, 8, num_layers=2, dropout=0.4)
  (fc): Linear(in_features=8, out_features=1, bias=True)
)

# Generating LSTM predictions

In [25]:
def generate_predictions(dataloader_pos, dataset_pos, net_pos, df_original, first_gw):
    index = 0
    predictions_pos = []
    for inputs in dataloader_pos:
        inputs = torch.flatten(inputs, 1, 2)
        inputs = inputs.unsqueeze(2).to(device)

        outputs = net_pos(inputs).to(device)
        # print("OUTPUTS", outputs)
        for output in outputs:
            player_id, sample_id = dataset_pos.index_table[index]
            name = dataset_pos.names[player_id]
            points = round(output.item(), 2)
            df_row = df_original[(df_original["Name"] == name) & (df_original["GW"] >= first_gw)].iloc[sample_id]
            df_row["Points"] = points
            df_gw = df_row["GW"]
            df_opp = df_row["Opponent"]
            
            gw = sample_id + FIS + 1 # wrong
            predictions_pos.append(df_row)
            index += 1
    
    output_df = pd.DataFrame(predictions_pos)
    output_df = output_df.loc[:, ~output_df.columns.str.contains('^Unnamed')]
    return output_df

In [26]:
prediction_columns = ["Name", "GW", "Opponent", "Was Home", "FPL", "Points"]

In [27]:
%%time
df_predictions_general = generate_predictions(dataloader_general_test, dataset_general, net, df_general, FROM_GW)

df_predictions_gk = generate_predictions(dataloader_gk_test, dataset_gk, net_gk, df_gk, FROM_GW)
df_predictions_def = generate_predictions(dataloader_def_test, dataset_def, net_def, df_def, FROM_GW)
df_predictions_mid = generate_predictions(dataloader_mid_test, dataset_mid, net_mid, df_mid, FROM_GW)
df_predictions_fwd = generate_predictions(dataloader_fwd_test, dataset_fwd, net_fwd, df_fwd, FROM_GW)

CPU times: total: 7min 34s
Wall time: 1min 58s


In [28]:
df_predictions_general[prediction_columns]

,Name,GW,Opponent,Was Home,FPL,Points
8166,Brenden-Aaronson,25.0,Southampton,1.0,NaN,2.81
8167,Brenden-Aaronson,26.0,Chelsea,0.0,NaN,2.81
8168,Brenden-Aaronson,27.0,Brighton,1.0,NaN,2.81
8169,Brenden-Aaronson,28.0,Wolves,0.0,NaN,2.81
8170,Brenden-Aaronson,29.0,Nott'ham Forest,1.0,NaN,2.81
...,...,...,...,...,...,...
11612,Martin-Odegaard,25.0,Leicester City,0.0,NaN,2.82
11613,Martin-Odegaard,26.0,Bournemouth,1.0,NaN,2.83
11614,Martin-Odegaard,27.0,Fulham,0.0,NaN,2.81
11615,Martin-Odegaard,28.0,Crystal Palace,1.0,NaN,2.82


# XGBoost models

In [29]:
xgb_features = ["Points", "Was Home", "Opp rating", 'ScoreForLast5', 'ScoreAgainstLast5']

# career averages
xgb_features += ['MinAvgOverall', 'GlsAvgOverall',
       'AstAvgOverall', 'CrdYAvgOverall', 'CrdRAvgOverall', 'xGAvgOverall',
       'xAGAvgOverall', 'CSAvgOverall', 'Team ScoreAvgOverall',
       'Opp ScoreAvgOverall', 'FPLAvgOverall']

info = ["Name", "FPL position", "GW", "Squad", "Opponent", "Was Home", "Opp rating"]

In [30]:
model_general = xgb.XGBRegressor()
model_general.load_model("models/xgb/model_general.json")

In [31]:
model_gk = xgb.XGBRegressor()
model_gk.load_model("models/xgb/model_gk.json")

In [32]:
model_def = xgb.XGBRegressor()
model_def.load_model("models/xgb/model_def.json")

In [33]:
model_mid = xgb.XGBRegressor()
model_mid.load_model("models/xgb/model_mid.json")

In [34]:
model_fwd = xgb.XGBRegressor()
model_fwd.load_model("models/xgb/model_fwd.json")

# Generating XGBoost predictions

In [35]:
X_test_general = df_predictions_general[xgb_features]

X_test_gk = df_predictions_gk[xgb_features]
X_test_def = df_predictions_def[xgb_features]
X_test_mid = df_predictions_mid[xgb_features]
X_test_fwd = df_predictions_fwd[xgb_features]

In [36]:
predictions_general = model_general.predict(X_test_general)

predictions_gk = model_gk.predict(X_test_gk)
predictions_def = model_def.predict(X_test_def)
predictions_mid = model_mid.predict(X_test_mid)
predictions_fwd = model_fwd.predict(X_test_fwd)

In [37]:
df_predictions_general["pred"] = predictions_general

df_predictions_gk["pred"] = predictions_gk
df_predictions_def["pred"] = predictions_def
df_predictions_mid["pred"] = predictions_mid
df_predictions_fwd["pred"] = predictions_fwd

# Printing predictions

In [38]:
df_predictions_general[info + ["pred"]].sort_values(by=["pred"], ascending = False).head(20)

,Name,FPL position,GW,Squad,Opponent,Was Home,Opp rating,pred
8996,Diego-Costa,FWD,28.0,Wolves,Leeds United,1.0,1708.115723,11.468849
9496,Gabriel-Jesus,FWD,27.0,Arsenal,Fulham,0.0,1717.072754,11.359003
11927,Mohamed-Salah,MID,29.0,Liverpool,Manchester City,0.0,2026.310791,11.177331
11627,Martinelli,MID,25.0,Arsenal,Leicester City,0.0,1746.499023,10.759837
8994,Diego-Costa,FWD,26.0,Wolves,Tottenham,1.0,1858.896484,10.744741
11630,Martinelli,MID,28.0,Arsenal,Crystal Palace,1.0,1742.434937,10.672922
11629,Martinelli,MID,27.0,Arsenal,Fulham,0.0,1717.072754,10.582842
9497,Gabriel-Jesus,FWD,28.0,Arsenal,Crystal Palace,1.0,1742.434937,9.823874
11631,Martinelli,MID,29.0,Arsenal,Leeds United,1.0,1708.115723,9.759139
11928,Mohamed-Salah,MID,29.0,Liverpool,Chelsea,0.0,1870.791016,9.617655


In [39]:
df_predictions_gk[info + ["pred"]].sort_values(by=["pred"], ascending = False).head(20)

,Name,FPL position,GW,Squad,Opponent,Was Home,Opp rating,pred
11584,Mark-Travers,GK,28.0,Bournemouth,Aston Villa,0.0,1766.583008,7.927203
8831,David-Raya,GK,27.0,Brentford,Everton,0.0,1667.462280,7.833431
11581,Mark-Travers,GK,25.0,Bournemouth,Manchester City,1.0,2026.310791,7.171827
11586,Mark-Travers,GK,29.0,Bournemouth,Brighton,1.0,1822.430054,7.153015
8830,David-Raya,GK,26.0,Brentford,Fulham,1.0,1717.072754,7.108498
8832,David-Raya,GK,27.0,Brentford,Southampton,0.0,1651.960449,7.046344
11583,Mark-Travers,GK,27.0,Bournemouth,Liverpool,1.0,1963.465210,6.716725
11582,Mark-Travers,GK,26.0,Bournemouth,Arsenal,0.0,1946.912109,6.682509
8786,Danny-Ward,GK,27.0,Leicester City,Chelsea,1.0,1870.791016,6.681073
7490,Aaron-Ramsdale,GK,27.0,Arsenal,Fulham,0.0,1717.072754,5.940970


In [40]:
df_predictions_def[info + ["pred"]].sort_values(by=["pred"], ascending = False).head(20)

,Name,FPL position,GW,Squad,Opponent,Was Home,Opp rating,pred
10945,Kieran-Trippier,DEF,28.0,Newcastle Utd,Nott'ham Forest,0.0,1655.643188,10.998822
10944,Kieran-Trippier,DEF,27.0,Newcastle Utd,Wolves,1.0,1695.581787,9.960417
12576,Reece-James,DEF,26.0,Chelsea,Leeds United,1.0,1708.115723,8.952859
12578,Reece-James,DEF,28.0,Chelsea,Everton,1.0,1667.462280,8.211574
10947,Kieran-Trippier,DEF,29.0,Newcastle Utd,West Ham,0.0,1748.305420,8.109373
8051,Ben-White,DEF,26.0,Arsenal,Bournemouth,1.0,1636.081665,6.930041
7787,Andrew-Robertson,DEF,27.0,Liverpool,Bournemouth,0.0,1636.081665,6.859110
9361,Fabian-Schar,DEF,28.0,Newcastle Utd,Nott'ham Forest,0.0,1655.643188,6.740617
7788,Andrew-Robertson,DEF,28.0,Liverpool,Fulham,1.0,1717.072754,6.656566
10561,Jonny-Evans,DEF,29.0,Leicester City,Aston Villa,1.0,1766.583008,6.477551


In [41]:
df_predictions_mid[info + ["pred"]].sort_values(by=["pred"], ascending = False).head(20)

,Name,FPL position,GW,Squad,Opponent,Was Home,Opp rating,pred
11631,Martinelli,MID,29.0,Arsenal,Leeds United,1.0,1708.115723,13.780984
11627,Martinelli,MID,25.0,Arsenal,Leicester City,0.0,1746.499023,12.719133
12770,Rodri,MID,27.0,Manchester City,Crystal Palace,0.0,1742.434937,11.218611
13420,Thomas-Partey,MID,25.0,Arsenal,Leicester City,0.0,1746.499023,10.918680
8360,Casemiro,MID,29.0,Manchester Utd,Newcastle Utd,0.0,1855.404907,10.501121
11613,Martin-Odegaard,MID,26.0,Arsenal,Bournemouth,1.0,1636.081665,9.881488
11865,Miguel-Almiron,MID,28.0,Newcastle Utd,Nott'ham Forest,0.0,1655.643188,9.567157
11864,Miguel-Almiron,MID,27.0,Newcastle Utd,Wolves,1.0,1695.581787,9.555065
8171,Brenden-Aaronson,MID,29.0,Leeds United,Arsenal,0.0,1946.912109,9.540378
8169,Brenden-Aaronson,MID,28.0,Leeds United,Wolves,0.0,1695.581787,9.313539


In [42]:
df_predictions_fwd[info + ["pred"]].sort_values(by=["pred"], ascending = False).head(20)

,Name,FPL position,GW,Squad,Opponent,Was Home,Opp rating,pred
9494,Gabriel-Jesus,FWD,25.0,Arsenal,Leicester City,0.0,1746.499023,18.336842
9496,Gabriel-Jesus,FWD,27.0,Arsenal,Fulham,0.0,1717.072754,18.313971
9497,Gabriel-Jesus,FWD,28.0,Arsenal,Crystal Palace,1.0,1742.434937,17.865231
9498,Gabriel-Jesus,FWD,29.0,Arsenal,Leeds United,1.0,1708.115723,14.557178
9493,Gabriel-Jesus,FWD,25.0,Arsenal,Everton,1.0,1667.462280,11.701508
9495,Gabriel-Jesus,FWD,26.0,Arsenal,Bournemouth,1.0,1636.081665,11.658823
7651,Aleksandar-Mitrovic,FWD,26.0,Fulham,Brentford,0.0,1779.055054,10.354334
9154,Eddie-Nketiah,FWD,25.0,Arsenal,Everton,1.0,1667.462280,9.942062
9157,Eddie-Nketiah,FWD,27.0,Arsenal,Fulham,0.0,1717.072754,9.059914
9158,Eddie-Nketiah,FWD,28.0,Arsenal,Crystal Palace,1.0,1742.434937,9.033876


# Assembly

In [43]:
df_predictions_positional_merged = pd.concat([df_predictions_gk, df_predictions_def, df_predictions_mid, df_predictions_fwd])

In [44]:
df_assembly = df_predictions_general

In [45]:
df_predictions_general.columns

Index(['Date', 'Day', 'Comp', 'Round', 'Venue', 'Squad', 'Opponent', 'Start',
       'Pos', 'Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY', 'CrdR',
       'Touches', 'Tkl', 'Int', 'Blocks', 'xG', 'npxG', 'xAG', 'SCA', 'GCA',
       'Cmp', 'Att', 'Cmp%', 'PrgP', 'Carries', 'PrgC', 'Att.1', 'Succ',
       'Match Report', 'Name', 'Season', 'WDL', 'GoalsTeams', 'Team Score',
       'Opp Score', 'CS', 'Was Home', 'GW', 'FPL', 'neutral_name',
       'Total Points', 'FPL position', 'Opp rating', 'Team rating',
       'ScoreForLast5', 'ScoreAgainstLast5', 'MinAvgOverall', 'GlsAvgOverall',
       'AstAvgOverall', 'CrdYAvgOverall', 'CrdRAvgOverall', 'xGAvgOverall',
       'xAGAvgOverall', 'CSAvgOverall', 'Team ScoreAvgOverall',
       'Opp ScoreAvgOverall', 'FPLAvgOverall', 'Finished', 'Squad H',
       'Squad A', 'Points', 'pred'],
      dtype='object')

In [46]:
for i, row in df_assembly.iterrows():
    name = row["Name"]
    gw = row["GW"]
    season = row["Season"]
    squadH = row["Squad H"]
    squadA = row["Squad A"]
    row_merged = df_predictions_positional_merged[(df_predictions_positional_merged["Name"] == name) & (df_predictions_positional_merged["GW"] == gw) & (df_predictions_positional_merged["Season"] == season) & (df_predictions_positional_merged["Squad H"] == squadH) & (df_predictions_positional_merged["Squad A"] == squadA)]
    
    if row_merged.shape[0] > 0:
        df_assembly.at[i, "general pred"] = row["pred"]
        df_assembly.at[i, "positional pred"] = row_merged["pred"].item()
        df_assembly.at[i, "assembly pred"] = round((row["pred"] + row_merged["pred"].item()) / 2, 2)
    
# dropping old pred column
df_assembly = df_assembly.drop(['pred'], axis=1)

In [47]:
df_assembly_sorted = df_assembly[info + ["assembly pred", "general pred", "positional pred"]].sort_values(by=["assembly pred"], ascending = False)
df_assembly_sorted.head(20)

,Name,FPL position,GW,Squad,Opponent,Was Home,Opp rating,assembly pred,general pred,positional pred
9496,Gabriel-Jesus,FWD,27.0,Arsenal,Fulham,0.0,1717.072754,14.84,11.359003,18.313971
9494,Gabriel-Jesus,FWD,25.0,Arsenal,Leicester City,0.0,1746.499023,13.85,9.357240,18.336842
9497,Gabriel-Jesus,FWD,28.0,Arsenal,Crystal Palace,1.0,1742.434937,13.84,9.823874,17.865231
11631,Martinelli,MID,29.0,Arsenal,Leeds United,1.0,1708.115723,11.77,9.759139,13.780984
11627,Martinelli,MID,25.0,Arsenal,Leicester City,0.0,1746.499023,11.74,10.759837,12.719133
9498,Gabriel-Jesus,FWD,29.0,Arsenal,Leeds United,1.0,1708.115723,10.72,6.890312,14.557178
7651,Aleksandar-Mitrovic,FWD,26.0,Fulham,Brentford,0.0,1779.055054,9.67,8.979160,10.354334
11630,Martinelli,MID,28.0,Arsenal,Crystal Palace,1.0,1742.434937,9.43,10.672922,8.194481
11865,Miguel-Almiron,MID,28.0,Newcastle Utd,Nott'ham Forest,0.0,1655.643188,9.38,9.196863,9.567157
9317,Erling-Haaland,FWD,27.0,Manchester City,Crystal Palace,0.0,1742.434937,9.01,9.207751,8.804855


In [48]:
df_assembly_sorted[df_assembly_sorted["Name"] == "Martinelli"]

,Name,FPL position,GW,Squad,Opponent,Was Home,Opp rating,assembly pred,general pred,positional pred
11631,Martinelli,MID,29.0,Arsenal,Leeds United,1.0,1708.115723,11.77,9.759139,13.780984
11627,Martinelli,MID,25.0,Arsenal,Leicester City,0.0,1746.499023,11.74,10.759837,12.719133
11630,Martinelli,MID,28.0,Arsenal,Crystal Palace,1.0,1742.434937,9.43,10.672922,8.194481
11629,Martinelli,MID,27.0,Arsenal,Fulham,0.0,1717.072754,8.78,10.582842,6.980433
11628,Martinelli,MID,26.0,Arsenal,Bournemouth,1.0,1636.081665,8.74,9.320693,8.158043
11626,Martinelli,MID,25.0,Arsenal,Everton,1.0,1667.462280,8.12,8.485924,7.758463


# Assembly sumed up for double gameweeks

In [49]:
df_assembly_sumed = df_assembly_sorted.groupby(['GW', 'Name', "FPL position", "Squad"]).sum().reset_index()

In [50]:
df_assembly_sorted.columns

Index(['Name', 'FPL position', 'GW', 'Squad', 'Opponent', 'Was Home',
       'Opp rating', 'assembly pred', 'general pred', 'positional pred'],
      dtype='object')

In [51]:
df_assembly_sumed.columns

Index(['GW', 'Name', 'FPL position', 'Squad', 'Was Home', 'Opp rating',
       'assembly pred', 'general pred', 'positional pred'],
      dtype='object')

In [52]:
df_assembly_sumed = df_assembly_sumed.sort_values(by=["assembly pred"], ascending = False)[["Name", "FPL position", "Squad", "GW", "assembly pred", "general pred", "positional pred"]]

In [53]:
df_assembly_sumed

,Name,FPL position,Squad,GW,assembly pred,general pred,positional pred
104,Gabriel-Jesus,FWD,Arsenal,25.0,22.79,15.541883,30.038349
223,Martinelli,MID,Arsenal,25.0,19.86,19.245761,20.477597
1928,Miguel-Almiron,MID,Newcastle Utd,29.0,16.82,16.375174,17.257722
1932,Mohamed-Salah,MID,Liverpool,29.0,16.26,20.794986,11.729902
1866,Kieran-Trippier,DEF,Newcastle Utd,29.0,15.23,16.509141,13.949657
...,...,...,...,...,...,...,...
21,Asmir-Begovic,GK,Everton,25.0,-0.27,-0.556467,0.000144
385,Ben-Pearson,MID,Bournemouth,26.0,-0.29,-0.811666,0.238315
1227,Angelo-Ogbonna,DEF,West Ham,28.0,-0.33,-0.593930,-0.065760
1602,Thilo-Kehrer,DEF,West Ham,28.0,-0.55,-0.743823,-0.360704


In [54]:
df_assembly_sumed[df_assembly_sumed["Name"] == "Martinelli"][["Name", "GW", "assembly pred", "general pred", "positional pred"]]

,Name,GW,assembly pred,general pred,positional pred
223,Martinelli,25.0,19.86,19.245761,20.477597
1912,Martinelli,29.0,11.77,9.759139,13.780984
1483,Martinelli,28.0,9.43,10.672922,8.194481
1054,Martinelli,27.0,8.78,10.582842,6.980433
625,Martinelli,26.0,8.74,9.320693,8.158043


# Assembly pivot

In [55]:
df_pivot = df_assembly_sumed.pivot(index=['Name', 'FPL position', 'Squad'], columns='GW', values='assembly pred').reset_index()
df_pivot = df_pivot.rename(columns=lambda x: "GW" + str(x) if x not in  ["Name", "FPL position", "Squad"] else x)
df_pivot = df_pivot.rename(columns=lambda x: x[:-2] if x.startswith('GW') else x)
df_pivot = df_pivot.fillna(0)
df_pivot = df_pivot.rename_axis(None, axis=1)
df_pivot['Sum'] = df_pivot.iloc[:, 1:].sum(axis=1)
df_pivot = df_pivot.sort_values('Sum', ascending=False)

C:\Users\user\AppData\Local\Temp\ipykernel_4160\189157183.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_pivot['Sum'] = df_pivot.iloc[:, 1:].sum(axis=1)


## Assembly pivot - every position

In [56]:
df_pivot.head(20)

,Name,FPL position,Squad,GW25,GW26,GW27,GW28,GW29,Sum
135,Gabriel-Jesus,FWD,Arsenal,22.79,8.90,14.84,13.84,10.72,71.09
278,Martinelli,MID,Arsenal,19.86,8.74,8.78,9.43,11.77,58.58
298,Mohamed-Salah,MID,Liverpool,13.31,6.43,5.88,7.42,16.26,49.30
277,Martin-Odegaard,MID,Arsenal,15.02,8.68,6.98,7.50,6.31,44.49
294,Miguel-Almiron,MID,Newcastle Utd,0.00,8.23,8.07,9.38,16.82,42.50
123,Erling-Haaland,FWD,Manchester City,6.24,7.68,9.01,8.68,7.79,39.40
101,Diego-Costa,FWD,Wolves,11.29,6.73,5.97,8.60,6.44,39.03
46,Brenden-Aaronson,MID,Leeds United,6.50,5.88,5.92,7.19,13.23,38.72
232,Kieran-Trippier,DEF,Newcastle Utd,0.00,5.39,7.89,8.45,15.23,36.96
139,Granit-Xhaka,MID,Arsenal,10.79,5.69,6.55,4.46,5.96,33.45


## Assembly pivot - goalkeepers

In [57]:
df_pivot[df_pivot["FPL position"] == "GK"].head(10)

,Name,FPL position,Squad,GW25,GW26,GW27,GW28,GW29,Sum
275,Mark-Travers,GK,Bournemouth,5.70,5.39,5.68,6.07,10.60,33.44
90,David-Raya,GK,Brentford,0.00,5.95,13.33,4.08,8.88,32.24
2,Aaron-Ramsdale,GK,Arsenal,6.51,4.67,4.93,3.60,4.34,24.05
93,Dean-Henderson,GK,Nott'ham Forest,3.09,4.07,3.05,3.10,10.73,24.04
136,Gavin-Bazunu,GK,Southampton,5.13,4.34,7.18,3.04,3.94,23.63
154,Illan-Meslier,GK,Leeds United,3.29,3.71,4.14,3.58,7.94,22.66
209,Jordan-Pickford,GK,Everton,7.74,3.22,5.05,2.80,3.36,22.17
87,Danny-Ward,GK,Leicester City,4.68,1.75,4.81,3.79,6.33,21.36
91,David-de-Gea,GK,Manchester Utd,0.00,2.57,4.74,3.71,8.57,19.59
257,Lukasz-Fabianski,GK,West Ham,3.98,2.25,3.23,2.20,6.41,18.07


## Assembly pivot - defenders

In [58]:
df_pivot[df_pivot["FPL position"] == "DEF"].head(10)

,Name,FPL position,Squad,GW25,GW26,GW27,GW28,GW29,Sum
232,Kieran-Trippier,DEF,Newcastle Utd,0.00,5.39,7.89,8.45,15.23,36.96
39,Ben-White,DEF,Arsenal,9.15,6.11,4.56,3.28,5.70,28.80
341,Reece-James,DEF,Chelsea,4.56,6.11,4.50,6.07,6.90,28.14
249,Lloyd-Kelly,DEF,Bournemouth,5.29,4.15,5.13,4.44,8.13,27.14
21,Andrew-Robertson,DEF,Liverpool,7.93,2.84,4.99,5.46,5.56,26.78
422,William-Saliba,DEF,Arsenal,7.16,6.10,4.40,2.01,5.40,25.07
326,Pascal-Struijk,DEF,Leeds United,5.34,2.68,2.24,4.35,8.67,23.28
134,Gabriel-Dos-Santos,DEF,Arsenal,6.31,4.82,3.39,2.79,4.26,21.57
172,James-Justin,DEF,Leicester City,2.73,3.97,2.37,3.76,8.59,21.42
247,Liam-Cooper,DEF,Leeds United,4.16,2.76,3.31,3.46,7.44,21.13


## Assembly pivot - midfielders

In [59]:
df_pivot[df_pivot["FPL position"] == "MID"].head(10)

,Name,FPL position,Squad,GW25,GW26,GW27,GW28,GW29,Sum
278,Martinelli,MID,Arsenal,19.86,8.74,8.78,9.43,11.77,58.58
298,Mohamed-Salah,MID,Liverpool,13.31,6.43,5.88,7.42,16.26,49.30
277,Martin-Odegaard,MID,Arsenal,15.02,8.68,6.98,7.50,6.31,44.49
294,Miguel-Almiron,MID,Newcastle Utd,0.00,8.23,8.07,9.38,16.82,42.50
46,Brenden-Aaronson,MID,Leeds United,6.50,5.88,5.92,7.19,13.23,38.72
139,Granit-Xhaka,MID,Arsenal,10.79,5.69,6.55,4.46,5.96,33.45
108,Douglas-Luiz,MID,Aston Villa,5.81,4.87,5.19,6.26,11.30,33.43
52,Bukayo-Saka,MID,Arsenal,11.15,4.86,6.21,6.01,4.82,33.05
146,Harvey-Barnes,MID,Leicester City,4.88,6.63,3.66,4.13,12.42,31.72
355,Rodrigo,MID,Leeds United,5.69,3.57,3.52,6.02,10.91,29.71


## Assembly pivot - forwards

In [60]:
df_pivot[df_pivot["FPL position"] == "FWD"].head(10)

,Name,FPL position,Squad,GW25,GW26,GW27,GW28,GW29,Sum
135,Gabriel-Jesus,FWD,Arsenal,22.79,8.90,14.84,13.84,10.72,71.09
123,Erling-Haaland,FWD,Manchester City,6.24,7.68,9.01,8.68,7.79,39.40
101,Diego-Costa,FWD,Wolves,11.29,6.73,5.97,8.60,6.44,39.03
12,Aleksandar-Mitrovic,FWD,Fulham,3.48,9.67,6.09,8.01,4.03,31.28
157,Ivan-Toney,FWD,Brentford,0.00,6.16,5.95,6.50,11.79,30.40
112,Eddie-Nketiah,FWD,Arsenal,9.86,4.73,5.29,4.64,5.26,29.78
321,Ollie-Watkins,FWD,Aston Villa,6.14,4.13,4.38,5.37,9.29,29.31
54,Callum-Wilson,FWD,Newcastle Utd,0.00,5.20,6.29,5.10,8.80,25.39
106,Dominic-Solanke,FWD,Bournemouth,4.49,4.35,4.18,4.03,7.99,25.04
62,Che-Adams,FWD,Southampton,4.15,3.16,7.61,4.01,3.36,22.29


# Saving final dataframes

In [62]:
df_pivot.to_csv("predictions/final/final_all_positions.csv")

df_pivot[df_pivot["FPL position"] == "GK"].to_csv("predictions/final/final_gk.csv")
df_pivot[df_pivot["FPL position"] == "DEF"].to_csv("predictions/final/final_def.csv")
df_pivot[df_pivot["FPL position"] == "MID"].to_csv("predictions/final/final_mid.csv")
df_pivot[df_pivot["FPL position"] == "FWD"].to_csv("predictions/final/final_fwd.csv")